# 1) Using CNN on input:  Rpeak -19 / +20  ticks  (QRS only, both channels)

In [1]:
import os, sys
sys.path.append("./module")
import cnn_ecg as cnn_ecg



## Load up Dataset #1 (Classification) and Dataset #2 (Evaluation) 

In [2]:
ds1_all, ds1_ann= cnn_ecg.create_DS("1")
ds2_all, ds2_ann= cnn_ecg.create_DS("2")
    

In [8]:
#ds1_ann["101"]['Type'].unique()
ds2_ann["222"]['Type'].unique()


array(['N', 'SVEB'], dtype=object)

In [69]:
vals_to_replace = {'N':'N','L':'N','e':'N','j':'N','R':'N','A':'SVEB','a':'SVEB','J':'SVEB','S':'SVEB','V':'VEB','E':'VEB','F':'F','Q':'Q','P':'Q','f':'Q','U':'Q'}
dfann['Type'] = dfann['Type'].map(vals_to_replace)
#dfann['Type'].replace(to_replace=['A', 'a'], value='SVEB' ,inplace=True)

In [70]:
dfann['Type'].unique()

array(['N', 'VEB'], dtype=object)